<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [120]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [121]:
import folium
import pandas as pd

In [122]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


## Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [123]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [124]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [125]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [126]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


In [127]:
# --- Offline fallback: build `launch_sites` + draw circles & labels on `site_map` ---
import pandas as pd, folium
from folium.features import DivIcon
from folium.plugins import MousePosition, MeasureControl

# 1) If `launch_sites` already exists, keep it; otherwise create it manually (offline)
if "launch_sites" not in globals():
    launch_sites = pd.DataFrame([
        {"Launch_Site": "CCAFS LC-40", "Lat": 28.562302, "Lon": -80.577356},
        {"Launch_Site": "KSC LC-39A",  "Lat": 28.573255, "Lon": -80.646895},
        {"Launch_Site": "VAFB SLC-4E", "Lat": 34.632834, "Lon": -120.610746},
        {"Launch_Site": "CCAFS SLC-40","Lat": 28.561857, "Lon": -80.577366},
    ])

# 2) Ensure the map object exists (your lab might name it `site_map` or `spacex_map`)
if "site_map" not in globals():
    center = [launch_sites["Lat"].mean(), launch_sites["Lon"].mean()]
    site_map = folium.Map(location=center, zoom_start=6, tiles="OpenStreetMap")
    MousePosition(position="bottomright", prefix="Lat/Lon:", separator=" | ").add_to(site_map)
    MeasureControl(primary_length_unit="kilometers").add_to(site_map)

# 3) Add a circle and a text marker for each site
for _, r in launch_sites.iterrows():
    coord = [float(r["Lat"]), float(r["Lon"])]

    # Circle with popup (radius 1 km)
    folium.Circle(
        location=coord,
        radius=1000,
        color="#d35400",
        fill=True,
        fill_color="#d35400",
        fill_opacity=0.15
    ).add_child(folium.Popup(r["Launch_Site"])).add_to(site_map)

    # Text label using DivIcon
    folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % r["Launch_Site"],
        ),
    ).add_to(site_map)

# 4) Show the map (must be last line in the cell)
site_map


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [128]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [129]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [130]:
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [131]:
# === TODO #1: Create marker_color column ===
# Safeguard name for the class column
cls_col = "class" if "class" in spacex_df.columns else "Class"

# Ensure numeric 0/1
spacex_df[cls_col] = spacex_df[cls_col].astype(int)

# Map to color
spacex_df["marker_color"] = spacex_df[cls_col].map({1: "green", 0: "red"})
spacex_df.head()


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [132]:
# === Draw an outline ring around the cluster of each launch site ===
from math import radians, sin, cos, asin, sqrt

def haversine_km(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return 6371 * (2 * asin(sqrt(a)))  # km

# pakai nama kolom yang sudah kamu pakai sebelumnya
site_c = "Launch Site" if "Launch Site" in spacex_df.columns else ("Launch_Site" if "Launch_Site" in spacex_df.columns else "site")
lat_c  = "Lat"  if "Lat"  in spacex_df.columns else "lat"
lon_c  = "Long" if "Long" in spacex_df.columns else ("Lon" if "Lon" in spacex_df.columns else "long")

for site, grp in spacex_df.dropna(subset=[site_c, lat_c, lon_c]).groupby(site_c):
    # pusat = rata-rata koordinat launch pada site tsb
    lat0 = float(grp[lat_c].mean())
    lon0 = float(grp[lon_c].mean())

    # radius = jarak terjauh dari pusat ke titik manapun + padding (meter)
    if len(grp) > 1:
        max_km = max(
            haversine_km(lon0, lat0, float(row[lon_c]), float(row[lat_c]))
            for _, row in grp.iterrows()
        )
        radius_m = int(max_km * 1000) + 120  # padding 120 m
    else:
        radius_m = 300  # fallback jika hanya 1 titik

    # Gambar ring (tanpa fill) supaya titik hijau/merah terlihat jelas
    folium.Circle(
        location=[lat0, lon0],
        radius=radius_m,
        color="#d35400",    # merah-oranye seperti contoh
        weight=3,
        fill=False
    ).add_to(site_map)

# tampilkan peta
site_map

Your updated map may look like the following screenshots:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


## TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [133]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [134]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [135]:
# === Task 3 - PART A: pick coastline point & compute distance ===
import folium

# 1) Pilih launch site yang ingin dihitung (harus ada di dataframe launch_sites dari Task 1)
ls_name = "CCAFS LC-40"   # <-- GANTI jika mau site lain: "KSC LC-39A" / "VAFB SLC-4E" / dst.
row = launch_sites[launch_sites["Launch_Site"] == ls_name].iloc[0]
launch_site_lat, launch_site_lon = float(row["Lat"]), float(row["Lon"])

# 2) Masukkan koordinat coastline dari HUD MousePosition (GANTI 2 angka ini)
coast_lat, coast_lon = 28.39220, -80.60770

# 3) Hitung jarak (km) pakai fungsi yang sudah ada di sel sebelumnya
distance_km = calculate_distance(launch_site_lat, launch_site_lon, coast_lat, coast_lon)

# 4) Tampilkan marker di titik coastline + info jarak
folium.Marker(
    location=[coast_lat, coast_lon],
    tooltip="Nearest coastline",
    popup=folium.Popup(f"Coastline<br><b>{distance_km:.2f} km</b> from {ls_name}", max_width=220),
    icon=folium.Icon(color="green")
).add_to(site_map)

# (opsional) ring tipis di sekitar site agar mirip contoh
folium.Circle([launch_site_lat, launch_site_lon], radius=800, color="#d35400", weight=2, fill=False).add_to(site_map)

site_map  # penting: baris terakhir menampilkan peta


In [136]:
# === Task 3 - PART B: draw line from site to coastline ===
folium.PolyLine(
    [[launch_site_lat, launch_site_lon], [coast_lat, coast_lon]],
    color="purple", weight=3, tooltip=f"{ls_name} → Coastline: {distance_km:.2f} km"
).add_to(site_map)

site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [137]:
# === Pilih site & titik pantai terdekat (pakai MousePosition) ===
ls_name = "CCAFS LC-40"   # atau site lain
row = launch_sites[launch_sites["Launch_Site"] == ls_name].iloc[0]
launch_site_lat, launch_site_lon = float(row["Lat"]), float(row["Lon"])

# >>> GANTI 2 angka di bawah dengan titik pantai TIMUR terdekat dari HUD MousePosition
coast_lat, coast_lon = 28.56320, -80.56050   # contoh saja; ambil milikmu sendiri

# Hitung jarak (KM)
distance_km = calculate_distance(launch_site_lat, launch_site_lon, coast_lat, coast_lon)

# === Gambar garis TIPIS & horizontal-ish seperti contoh ===
coordinates = [[launch_site_lat, launch_site_lon],
               [coast_lat,      coast_lon]]

# Garis tipis warna default (seperti screenshot referensi)
line = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(line)

# Label kecil “x.xx KM” di ujung sisi pantai (kanan)
folium.map.Marker(
    [coast_lat, coast_lon],
    icon=folium.DivIcon(
        html=f'<div style="color:#d35400;font-size:10px;"><b>{distance_km:.2f} KM</b></div>',
        icon_size=(0,0), icon_anchor=(0,0)
    )
).add_to(site_map)

site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [138]:
# === Railway (use course icon URL with fallback) ===
rail_lat, rail_lon = 28.57210, -80.58500   # <-- GANTI dengan koordinat railway terdekat

# Hitung jarak
rail_km = calculate_distance(launch_site_lat, launch_site_lon, rail_lat, rail_lon)

# Coba pakai CustomIcon dari URL; fallback ke FontAwesome 'train'
try:
    from folium.features import CustomIcon
    rail_icon = CustomIcon(
        icon_image="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png",
        icon_size=(24, 24),
        icon_anchor=(12, 12)
    )
except Exception:
    rail_icon = folium.Icon(color="white", icon_color="#1f77b4", icon="train", prefix="fa")

# Marker + garis
folium.Marker([rail_lat, rail_lon], tooltip="Nearest railway",
              popup=folium.Popup(f"Railway<br><b>{rail_km:.2f} km</b> from {ls_name}", max_width=200),
              icon=rail_icon).add_to(site_map)
folium.PolyLine([[launch_site_lat, launch_site_lon], [rail_lat, rail_lon]],
                color="#1f77b4", weight=1).add_to(site_map)

# Label jarak kecil di ujung garis
folium.map.Marker([rail_lat, rail_lon],
    icon=folium.DivIcon(html=f'<div style="color:#d35400;font-size:10px;"><b>{rail_km:.2f} KM</b></div>',
                        icon_size=(0,0), icon_anchor=(0,0))
).add_to(site_map)

site_map


In [139]:
# === Highway (use course icon URL with fallback) ===
hwy_lat, hwy_lon = 28.56450, -80.57680     # <-- GANTI dengan koordinat highway terdekat

hwy_km = calculate_distance(launch_site_lat, launch_site_lon, hwy_lat, hwy_lon)

try:
    from folium.features import CustomIcon
    hwy_icon = CustomIcon(
        icon_image="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png",
        icon_size=(24, 24),
        icon_anchor=(12, 12)
    )
except Exception:
    hwy_icon = folium.Icon(color="white", icon_color="#ff7f0e", icon="road", prefix="fa")

folium.Marker([hwy_lat, hwy_lon], tooltip="Nearest highway",
              popup=folium.Popup(f"Highway<br><b>{hwy_km:.2f} km</b> from {ls_name}", max_width=200),
              icon=hwy_icon).add_to(site_map)
folium.PolyLine([[launch_site_lat, launch_site_lon], [hwy_lat, hwy_lon]],
                color="#ff7f0e", weight=1).add_to(site_map)

folium.map.Marker([hwy_lat, hwy_lon],
    icon=folium.DivIcon(html=f'<div style="color:#d35400;font-size:10px;"><b>{hwy_km:.2f} KM</b></div>',
                        icon_size=(0,0), icon_anchor=(0,0))
).add_to(site_map)

site_map


In [140]:
# === City (use course icon URL with fallback) ===
city_lat, city_lon = 28.39220, -80.60770   # <-- GANTI dengan koordinat city terdekat

city_km = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

try:
    from folium.features import CustomIcon
    city_icon = CustomIcon(
        icon_image="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png",
        icon_size=(24, 24),
        icon_anchor=(12, 12)
    )
except Exception:
    # 'building' icon as a close match
    city_icon = folium.Icon(color="white", icon_color="#d62728", icon="building", prefix="fa")

folium.Marker([city_lat, city_lon], tooltip="Nearest city",
              popup=folium.Popup(f"City<br><b>{city_km:.2f} km</b> from {ls_name}", max_width=200),
              icon=city_icon).add_to(site_map)
folium.PolyLine([[launch_site_lat, launch_site_lon], [city_lat, city_lon]],
                color="#d62728", weight=1, dash_array="5,7").add_to(site_map)

folium.map.Marker([city_lat, city_lon],
    icon=folium.DivIcon(html=f'<div style="color:#d35400;font-size:10px;"><b>{city_km:.2f} KM</b></div>',
                        icon_size=(0,0), icon_anchor=(0,0))
).add_to(site_map)

site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


In [141]:
import pandas as pd

data = [
    # GANTI contoh di bawah dengan variabel jarakmu
    {"Launch Site": ls_name, "To Coast (km)": round(distance_km,2),
     "To Railway (km)": round(rail_km,2), "To Highway (km)": round(hwy_km,2),
     "To City (km)": round(city_km,2)}
]
pd.DataFrame(data)


,Launch Site,To Coast (km),To Railway (km),To Highway (km),To City (km)
0,CCAFS LC-40,1.65,1.32,0.25,19.15


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
